In [86]:
import scipy.integrate as spi
from scipy.optimize import minimize
from scipy.stats import norm
from scipy.stats import moment
import numpy as np

# Проверка гипотезы с помощью критерия хи-квадрат

Считаем параметры a и sigma для нормального распределения

In [87]:
def norm_distribution(x, a, sigma):
    return 1 / ((2 * np.pi) ** 0.5 * sigma) * np.exp(-((x - a) / sigma) ** 2 / 2)

def ln_p(a, b, x):
    if a == 0:
        a = -np.inf
    if b == 10:
        b = np.inf
    value = (spi.quad(norm_distribution, a, b, (x[0], x[1])))[0]
    if (value <= 0):
        return -10
    return np.log(value)

def ln_L(x):
    return -(5 * ln_p(0, 1, x) + 8 * ln_p(1, 2, x) + 6 * ln_p(2, 3, x) + 12 * ln_p(3, 4, x) + \
             14 * ln_p(4, 5, x) + 18 * ln_p(5, 6, x) + 11 * ln_p(6, 7, x) + 6 * ln_p(7, 8, x) + \
             13 * ln_p(8, 9, x) + 7 * ln_p(9, 10, x))

x0 = np.array([0.5, 0.5])
answer = minimize(ln_L, x0)
print('a =', answer.x[0])
print('sigma =', answer.x[1])

a = 5.289677406355167
sigma = 2.6795194456320277


Считаем дельта оценочное

In [88]:
def p(a, b, x):
    if a == 0:
        a = -np.inf
    if b == 10:
        b = np.inf
    return (spi.quad(norm_distribution, a, b, (x[0], x[1])))[0]

n = 100
m_i = [5, 8, 6, 12, 14, 18, 11, 6, 13, 7]
np_i = [p(i, (i + 1), answer.x) * n for i in range(len(m_i))]

delta_est = sum((m_i[i] - np_i[i]) ** 2 / np_i[i] for i in range(len(m_i)))

print("delta_est =", delta_est)

delta_est = 9.802553957084529


# Проверка гипотезы с помощью критерия Колмогорова

In [89]:
def F_empirical(x_n, x):
    return sum([1 if i < x else 0 for i in x_n]) / len(x_n)

def delta(x_n):
    ans = 0
    a = np.mean(x_n)
    sigma_sq = len(x_n) / (len(x_n) - 1) * moment(x_n, moment=2)
    for i in range(10):
        ans = max(ans, abs(norm.cdf(i, loc=a, scale=(sigma_sq) ** 0.5) - F_empirical(x_n, i)), abs(norm.cdf(i, loc=a, scale=sigma_sq ** 0.5) - F_empirical(x_n, i + 1)))
    return ans * len(x_n) ** 0.5

def bootstrap_distribution(n, x_n):
    delta_est = delta(x_n)
    a = np.mean(x_n)
    sigma_sq = len(x_n) / (len(x_n) - 1) * moment(x_n, moment=2)
    delta_i = []
    for i in range(50000):
        sample = [int(x_i) for x_i in norm.rvs(loc=a, scale=sigma_sq ** 0.5, size=n)]
        delta_i.append(delta(sample))
    sorted(delta_i)
    p_value = sum([1 if i >= delta_est else 0 for i in delta_i]) / 50000

    print("delta_est =", delta_est)
    print("p-value =", p_value)

A_i = [5, 8, 6, 12, 14, 18, 11, 6, 13, 7]
x_n = []
for i in range(10):
    x_n += [i] * A_i[i]
bootstrap_distribution(100, x_n)

delta_est = 1.0020942299329128
p-value = 0.79802
